# Data Wrangling: Join, Combine, and Reshape

##### Focues on ***combine, join, and rearrange*** data

In [1]:
import pandas as pd
import numpy as np

## 1. Hierarchical Indexing  
**multiple**(two or more) index levels on the axis

In [10]:
data = pd.Series(np.random.randn(9),
                 index = [['a','a','a','b','b','c','c','d','d'],list('123131223')])

In [11]:
data

a  1   -0.595939
   2    0.577280
   3   -0.353964
b  1    0.516423
   3   -0.610737
c  1   -0.010285
   2   -0.835307
d  2    1.249767
   3    0.000324
dtype: float64

In [12]:
data.index

MultiIndex([('a', '1'),
            ('a', '2'),
            ('a', '3'),
            ('b', '1'),
            ('b', '3'),
            ('c', '1'),
            ('c', '2'),
            ('d', '2'),
            ('d', '3')],
           )

*Partial* indexing:

In [14]:
data['b':'c']

b  1    0.516423
   3   -0.610737
c  1   -0.010285
   2   -0.835307
dtype: float64

In [16]:
data.loc[['b','d']]

b  1    0.516423
   3   -0.610737
d  2    1.249767
   3    0.000324
dtype: float64

In [46]:
data.loc[:, '2']
#selection from a inner level

a    0.577280
c   -0.835307
d    1.249767
dtype: float64

>unstack()

Rearrange the data into a DataFrame
>stack()

The inverse operation of the above

In [28]:
data.unstack()

,1,2,3
a,-0.595939,0.577280,-0.353964
b,0.516423,NaN,-0.610737
c,-0.010285,-0.835307,NaN
d,NaN,1.249767,0.000324


In [29]:
data.unstack().stack()

a  1   -0.595939
   2    0.577280
   3   -0.353964
b  1    0.516423
   3   -0.610737
c  1   -0.010285
   2   -0.835307
d  2    1.249767
   3    0.000324
dtype: float64

With a DataFrame, either axis can have a hierarchical index:

In [30]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                     index = [['a','a','b','b'], [1,2,1,2]],
                     columns = [['Ohio', 'Ohio', 'Colorado'],['Green', 'Red', 'Green']])

In [31]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

The hierarchical levels can have **names**:

In [36]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']

In [37]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [38]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

### 1.1 Reordering and Sorting Levels  
reagrrange the order of the levels on an axis, or sort the data by values in one specific level

>swaplevel()

In [41]:
frame.swaplevel('key1', 'key2')
#frame.swaplevel(0,1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

>sort_index(level=0)

Using values in a **single** level

In [43]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [44]:
frame.swaplevel(0,1).sort_index()

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## 2. Combining and Merging Datasets

Data can be combined together in a number of ways:
>pandas.merge  
>pandas.concat  
>combine_first

### 2.1 Database-Style DataFrame Joins  
*Merge* or *join* operations combine datasets by linking rows using one or more keys.

Use **on argument** to specify keys:  
>pandas.merge(df1, df2, on='key')  

It can be specified **separately**:  
>pd.merge(df1, df2, left_on='key1', right_on='key2')  

To merge woth multiple keys, pass a **list of column names**:  
>pd.merge(df1, df2, on=['key1', 'key2'])

In [15]:
df1 = pd.DataFrame({'key':list('bbacaab'),
                    'data1': range(7)})

In [16]:
df2 = pd.DataFrame({'key': list('abd'),
                    'data2': range(3)})

In [12]:
pd.merge(df1, df2, on='key')


,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


By default *merge* does a n 'inner' join. **how argument** provides options like: 'left', 'right', 'outer'  
>pd.merge(df1,df2, how='inner')

In [19]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


### 2.2 Merging on Index  
When merge key(s) are in **Index**:  
>left_index=True or right_index=True

In [23]:
pd.merge(df1, df2, suffixes=('_df1', '_df2'), left_index=True, right_index=True)
# suffixes optoion specify strings to append to overlapping names

,key_df1,data1,key_df2,data2
0,b,0,a,0
1,b,1,b,1
2,a,2,d,2


### 2.3 Concatenating Along an Axis

Calling objects in a **list**  
>pandas.concat([s1, s2], axis = 0)  

Using **key argument** to indentify the result:
>pandas.concat([s1, s2], keys = ['name1', 'name2'])

In [28]:
s1 = pd.Series([0, 1], index = ['a', 'b'])
s2 = pd.Series([2, 3, 4], index = ['c', 'd', 'e'])
s3 = pd.Series([5, 6], index = ['f', 'g'])

In [33]:
s4 = pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])
s4

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

When axis=1, this keys became the DataFrame colummn headers:
>pd.concat([s1, s2], axis=1, keys=['one', 'two'])

In [37]:
pd.concat([s1, s2], axis=1, keys=['one', 'two'], sort=True)

,one,two
a,0.0,NaN
b,1.0,NaN
c,NaN,2.0
d,NaN,3.0
e,NaN,4.0


When **row index** does not contain any relevant data:
>pd.concat([s1, s2], ignore_index=True)

In [38]:
df1 = pd.DataFrame(np.random.randn(2,2), columns = ['a', 'b'])
df2 = pd.DataFrame(np.random.randn(2,2), columns = ['b', 'c'])

In [42]:
pd.concat([df1, df2], sort=True)
#index is repeating below

,a,b,c
0,0.901610,-1.083199,NaN
1,-0.387768,0.223432,NaN
0,NaN,-0.923941,-0.200202
1,NaN,0.457090,0.450482


In [41]:
pd.concat([df1, df2], sort=True, ignore_index=True)

,a,b,c
0,0.901610,-1.083199,NaN
1,-0.387768,0.223432,NaN
2,NaN,-0.923941,-0.200202
3,NaN,0.457090,0.450482


### 2.4 Combining Data With Overlap  

**Patching missing data** in the calling object with data from the object u pass
>s1.combine_first(s2)

In [47]:
a = pd.Series([np.nan, 2.5, np.nan, 3.4, 4.5, np.nan])
b = pd.Series(np.arange(len(a)), dtype=np.float64)
b.iloc[-1] = np.nan

In [48]:
a.combine_first(b)

0    0.0
1    2.5
2    2.0
3    3.4
4    4.5
5    NaN
dtype: float64

In [49]:
b[:-2].combine_first(a[2:])

0    0.0
1    1.0
2    2.0
3    3.0
4    4.5
5    NaN
dtype: float64

In [50]:
b[:-2]

0    0.0
1    1.0
2    2.0
3    3.0
dtype: float64